## **CLASE 2: Middleware, rutas protegidas y autorización basada en roles**

### Objetivos de aprendizaje:

* Proteger rutas con `Depends` en FastAPI usando JWT.
* Controlar acceso según el rol del usuario (`admin`, `user`).
* Implementar middleware personalizado y pruebas de acceso.

---


### Contenidos y desarrollo


#### Árbol de carpetas para el proyecto FastAPI

```plaintext
my_project/
│
├── app/
│   ├── main.py
│   ├── config.py
│
│   ├── auth/
│   │   ├── jwt_handler.py         # Creación de tokens JWT
│   │   ├── security.py            # Validación y extracción del usuario actual
│   │   └── roles.py               # Verificación de rol por usuario
│
│   ├── routers/
│   │   ├── users.py               # Ruta protegida para obtener datos del usuario
│   │   └── admin.py               # Ruta protegida para administradores
│
│   ├── middleware/
│   │   └── validate_token.py      # Middleware que exige token en rutas /admin

```

---



#### 1. **Definir las variables de entorno**

app/config.py

In [ ]:
SECRET_KEY = "clave_super_segura"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30


#### 2. **Autenticación básica y creación de JWT seguro**

app/auth/jwt_handler.py

In [ ]:
from jose import JWTError, jwt
from datetime import datetime, timedelta
from app.config import SECRET_KEY, ALGORITHM, ACCESS_TOKEN_EXPIRE_MINUTES

def create_access_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.now() + (expires_delta or timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)


> **Buenas prácticas**:
>
> * Nunca exponer la clave secreta (`SECRET_KEY`) en el código fuente.
> * Establecer expiración del token.

---


#### 3. **Rutas protegidas y extracción del token con `Depends`**

#### Conceptos clave previos

| Concepto                 | Descripción                                                                                                                                 |
| ------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **JWT (JSON Web Token)** | Es un token seguro que contiene datos como el usuario y su rol. Se firma digitalmente y tiene fecha de expiración.                          |
| **`Depends` de FastAPI** | Mecanismo para declarar dependencias (como funciones reutilizables) que pueden validar, transformar o autorizar peticiones automáticamente. |
| **OAuth2PasswordBearer** | Clase de FastAPI que extrae automáticamente el token desde el encabezado HTTP `Authorization: Bearer <token>`.                              |

---


#### Paso a paso del código



* **Importaciones**


app/auth/security.py

In [ ]:
# Importa utilidades de FastAPI necesarias para la seguridad y manejo de errores
from fastapi import Depends, HTTPException, status

# Importa el esquema de autenticación OAuth2 con flujo "password"
from fastapi.security import OAuth2PasswordBearer

# Importa funciones para trabajar con JSON Web Tokens (JWT)
from jose import JWTError, jwt

# Importa las constantes SECRET_KEY y ALGORITHM desde el archivo de configuración de la app
from app.config import SECRET_KEY, ALGORITHM


* **Importamos los módulos**:

  * `Depends`: para inyectar funciones como dependencias.
  * `OAuth2PasswordBearer`: para extraer el token desde el encabezado.
  * `jwt.decode`: para leer el contenido del token.
  * `HTTPException`: para lanzar errores controlados.

---


* **Crear esquema de extracción del token**



In [ ]:
# Define el esquema de autenticación OAuth2 utilizando el flujo "password"
# El parámetro tokenUrl indica la URL del endpoint donde los usuarios enviarán sus credenciales para obtener un token
# En este caso, se espera que exista un endpoint POST en "/login" que genere el JWT
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")


* Esto configura FastAPI para esperar un token JWT en el **encabezado Authorization** usando el esquema Bearer:

  ```http
  Authorization: Bearer <token>
  ```
* `tokenUrl="login"` le dice a FastAPI que el endpoint de autenticación (que genera el token) está en `/login`.

---


* **Crear función de dependencia para obtener usuario actual**



In [ ]:
# Función que obtiene el usuario actual a partir del token JWT enviado en la solicitud
# Usa Depends para obtener automáticamente el token desde el esquema oauth2_scheme
def get_current_user(token: str = Depends(oauth2_scheme)):
    try:
        # Decodifica el token JWT usando la clave secreta y el algoritmo configurado
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])

        # Extrae el nombre de usuario desde el campo 'sub' del token
        user = payload.get("sub")

        # Extrae el rol del usuario desde el campo 'role' del token
        role = payload.get("role")

        # Si no se encuentra el usuario o el rol en el token, se lanza una excepción de credenciales
        if user is None or role is None:
            raise credentials_exception()

        # Si todo está correcto, retorna un diccionario con la información del usuario
        return {"username": user, "role": role}

    # Si ocurre un error al decodificar el token (por ejemplo, token inválido o expirado)
    except JWTError:
        raise credentials_exception()


#### ¿Qué hace esta función?

1. **Extrae el token** automáticamente del encabezado usando `Depends(oauth2_scheme)`.
2. **Decodifica** el token con la clave secreta y el algoritmo configurado.
3. Verifica que en el `payload` haya un `sub` (username) y un `role`.
4. Si el token es inválido o no tiene esos datos, lanza una excepción `401 Unauthorized`.

> Esta función puede ser utilizada como dependencia en cualquier endpoint que requiera autenticación.

---


* **Manejo de errores personalizados**



In [ ]:
# Función que retorna una excepción HTTP 401 para indicar que las credenciales son inválidas o no se proporcionaron
def credentials_exception():
    return HTTPException(
        # Código de estado HTTP 401: No autorizado
        status_code=status.HTTP_401_UNAUTHORIZED,

        # Mensaje de detalle que será enviado en la respuesta
        detail="Credenciales inválidas",

        # Encabezado que indica que se espera un token tipo Bearer en la autenticación
        headers={"WWW-Authenticate": "Bearer"},
    )



* Lanza un error `401 Unauthorized` con mensaje claro.
* El header `WWW-Authenticate` es estándar para OAuth2 y permite a clientes como SwaggerUI solicitar token nuevamente.

---


#### 4. **Autorización basada en roles**

app/auth/roles.py

In [ ]:
# Importa las herramientas necesarias de FastAPI
from fastapi import Depends, HTTPException

# Importa la función que obtiene al usuario actual desde el token JWT
from app.auth.security import get_current_user

# Función de orden superior que recibe un rol requerido y devuelve una dependencia
def require_role(required_role: str):

    # Esta función interna se usará como dependencia en los endpoints protegidos
    # Obtiene automáticamente al usuario actual a partir del token
    def role_dependency(user: dict = Depends(get_current_user)):

        # Compara el rol del usuario con el rol requerido
        if user["role"] != required_role:
            # Si el rol no coincide, lanza una excepción con código 403 (Prohibido)
            raise HTTPException(status_code=403, detail="Acceso denegado")

        # Si el rol es válido, permite continuar y retorna el usuario
        return user

    # Retorna la función como una dependencia que puede usarse en rutas protegidas
    return role_dependency


---

#### 5. **Middleware personalizado para validación de token**

app/middleware/validate_token.py

In [ ]:
# Importa la clase base para crear middlewares personalizados en Starlette (que FastAPI usa internamente)
from starlette.middleware.base import BaseHTTPMiddleware

# Importa el tipo Request para manejar solicitudes entrantes
from starlette.requests import Request

# Importa JSONResponse para poder devolver respuestas en formato JSON
from fastapi.responses import JSONResponse

# Define una clase de middleware personalizada que hereda de BaseHTTPMiddleware
class ValidateTokenMiddleware(BaseHTTPMiddleware):

    # Método principal que intercepta cada solicitud
    async def dispatch(self, request: Request, call_next):

        # Verifica si la ruta solicitada comienza con "/admin"
        if request.url.path.startswith("/admin"):

            # Obtiene el token desde el encabezado Authorization
            token = request.headers.get("Authorization")

            # Si no se encuentra el token, se responde con un error 401 (no autorizado)
            if token is None:
                return JSONResponse(
                    status_code=401,
                    content={"detail": "Token requerido"}
                )

        # Si todo está bien o la ruta no es "/admin", continúa con la solicitud normal
        return await call_next(request)


---

#### 6. **Ruta protegida para obtener datos del usuario**

app/routers/users.py

In [ ]:
# Importa APIRouter para definir rutas agrupadas en FastAPI
# Importa Depends para declarar dependencias (en este caso, obtener el usuario autenticado)
from fastapi import APIRouter, Depends

# Importa la función que valida el token y obtiene al usuario actual
from app.auth.security import get_current_user

# Crea un enrutador (puede ser incluido luego en la aplicación principal)
router = APIRouter()

# Define un endpoint GET en la ruta /datos-usuario
# Este endpoint requiere que el usuario esté autenticado
@router.get("/datos-usuario")
def get_user_data(current_user: dict = Depends(get_current_user)):
    # Retorna un diccionario con el nombre de usuario y el rol extraído del token
    return {
        "usuario_actual": current_user["username"],
        "rol": current_user["role"]
    }


---

#### 7. **Ruta protegida para administradores**

app/routers/admin.py

In [ ]:
# Importa APIRouter para definir rutas organizadas (modularización)
# Importa Depends para usar inyección de dependencias (en este caso, para verificar el rol del usuario)
from fastapi import APIRouter, Depends

# Importa la función que define la verificación del rol requerido
from app.auth.roles import require_role

# Crea un enrutador que puede incluirse en la aplicación principal
router = APIRouter()

# Define un endpoint GET en la ruta /admin-data
# Solo permitirá el acceso a usuarios con el rol "admin"
@router.get("/admin-data")
def get_admin_data(current_user=Depends(require_role("admin"))):
    # Si el usuario tiene rol "admin", retorna este mensaje
    return {"msg": "Datos solo para administradores"}


app/main.py

In [ ]:
# Importa FastAPI, el objeto principal para crear la aplicación
from fastapi import FastAPI

# Importa los routers definidos en los archivos users.py y admin.py
from app.routers import users, admin

# Importa el middleware personalizado que valida la presencia de un token para rutas protegidas
from app.middleware.validate_token import ValidateTokenMiddleware

# Crea una instancia de la aplicación FastAPI
app = FastAPI()

# Agrega el middleware personalizado a la aplicación
# Este middleware intercepta solicitudes antes de llegar a los endpoints
# En este caso, revisa si existe un token en los headers para rutas que comienzan con "/admin"
app.add_middleware(ValidateTokenMiddleware)

# Registra el router del módulo de usuarios (por ejemplo: /datos-usuario)
app.include_router(users.router)

# Registra el router del módulo de administrador (por ejemplo: /admin-data)
app.include_router(admin.router)


---

#### 8. **Instala las dependencias**

Desde la raíz del proyecto (`my_project/`), crea el archivo `requirements.txt` con lo siguiente:

```txt
fastapi
uvicorn
python-jose[cryptography]
passlib[bcrypt]
```

Luego, instala todo:

```bash
pip install -r requirements.txt
```

---

#### 9. **Ejecutar la aplicación**

Desde la raíz del proyecto:

```bash
uvicorn app.main:app --reload
```

Esto:

* Levanta el servidor en `http://127.0.0.1:8000`
* El parámetro `--reload` reinicia el servidor automáticamente si haces cambios

---

#### 10. **Simular un token JWT**

Como aún no hay un endpoint de login real, puedes **simular un token manualmente** desde Python:

```python
# ejecuta esto en un archivo Python aparte o consola interactiva
from jose import jwt
from datetime import datetime, timedelta

SECRET_KEY = "clave_super_segura"
ALGORITHM = "HS256"

data = {
    "sub": "juanperez",
    "role": "admin",  # o "user"
    "exp": datetime.utcnow() + timedelta(minutes=30)
}

token = jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)
print(token)
```

Copia el token generado y úsalo en Postman con:

```
Authorization: Bearer <token_aquí>
```

---

#### 11. **Probar desde Postman**

Ahora que tienes un token válido, puedes usarlo para probar tus rutas protegidas.

Abre Postman y sigue estos pasos:

1. Crear una solicitud `GET`

Por ejemplo, para el endpoint:

```
GET http://127.0.0.1:8000/datos-usuario
```

---

2. Ir a la pestaña **Authorization**

* Tipo: **Bearer Token**
* En el campo "Token", pega el token (solo el valor, sin escribir "Bearer")

Ejemplo:

```
eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...
```

---

3. Haz clic en **Send**

Postman enviará la petición con el token en el encabezado:

```
Authorization: Bearer <token>
```

---

Resultado esperado

Si el token es válido:

* Respuesta 200 OK
* Verás algo como:

```json
{
  "usuario_actual": "juanperez",
  "rol": "admin"
}
```



#### 12. **Pruebas que puedes hacer**

| Escenario                                   | Qué probar              | Resultado esperado       |
| ------------------------------------------- | ----------------------- | ------------------------ |
| Acceder a `/datos-usuario` sin token        | Lanza error 401         | "Credenciales inválidas" |
| Acceder a `/datos-usuario` con token válido | Devuelve username y rol | 200 OK                   |
| Acceder a `/admin-data` con rol `admin`     | Acceso permitido        | 200 OK                   |
| Acceder a `/admin-data` con rol `user`      | Lanza error 403         | "Acceso denegado"        |
| Acceder a `/admin-data` sin token           | Middleware lanza 401    | "Token requerido"        |

---


## Buenas prácticas de seguridad

| Recomendación                                    | Justificación                                              |
| ------------------------------------------------ | ---------------------------------------------------------- |
| Validar `exp` (expiración) del token             | Evita el uso de tokens viejos.                             |
| Usar HTTPS en producción                         | Protege el token en tránsito.                              |
| No incluir información sensible en el token      | El contenido de JWT puede ser leído por cualquier persona. |
| Usar `sub`, `role`, y `exp` como claims estándar | Facilita la interoperabilidad y mantenimiento del código.  |

---
